# Building a Panel Dashboard with Snowpark for Python

# Import needed modules

In [ ]:
# import snowpark for Python
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
# I saved all my credentials in a separete file and import them here,
# please use your own credentials
from snowflake_credentials import account, user, password, role, warehouse, database, schema

connection_parameters = {
    "account": account,
    "user": user,
    "password": password,
    "role": role,
    "warehouse": warehouse,
    "database": database,
    "schema": schema
}

# Get data

In [ ]:
session = Session.builder.configs(connection_parameters).create()
df_data = session.table('openstreetmap.pointsofinterest_nodes_usa_latlon_v1_quarterly_v1')
df_geo = session.sql('SELECT *, st_x(GEOM) as longitude, st_y(GEOM) as latitude FROM openstreetmap.geography_usa_latlon_v1')


In [ ]:
df_data.count(), df_geo.count()

In [ ]:
df = df_data.join(df_geo, df_data.col("geoid") == df_geo.col("geoid"))

In [ ]:
df.show(1)

In [ ]:
df.explain()

In [ ]:
str(df._plan.queries[0].sql)

In [ ]:
df.count()

In [ ]:
df = df.to_pandas()

In [ ]:
df.head(1)

# Plot 5 million data points with Datashader (hvplot rasterize=True)

In [ ]:
import holoviews as hv, pandas as pd, colorcet as cc
from holoviews.element.tiles import EsriImagery
import hvplot.pandas
from datashader.utils import lnglat_to_meters
import panel as pn
hv.extension('bokeh')

In [ ]:
# convert longitude and latitude
df.loc[:, 'x'], df.loc[:, 'y'] = lnglat_to_meters(df.LONGITUDE, df.LATITUDE)

In [ ]:
# plot 5 million rows and overlay with a map
map_tiles = EsriImagery().opts(alpha=0.5, width=700, height=480, bgcolor='black')
plot = df.hvplot(
    'x', 
    'y', 
    kind='scatter', 
    rasterize=True, 
    cmap=cc.fire, 
    cnorm='eq_hist',  
    colorbar=True).opts(colorbar_position='bottom')
map_tiles * plot

# Create an interactive dashboard

In [ ]:
# create a Panel widget to select top 10 amenities
select_amenity = pn.widgets.Select(
    options=df.AMENITY.value_counts().head(10).index.tolist(),
    name='Amenity'
)

In [ ]:
select_amenity

In [ ]:
# create a Panel dashboard
dfi = df.interactive
iplot = dfi[dfi.AMENITY==select_amenity].hvplot(
    'x',
    'y', 
    kind='scatter', 
    height=400,
    rasterize=True, 
    cmap=cc.fire, 
    cnorm='eq_hist', 
    colorbar=True).opts(colorbar_position='bottom')
map_tiles.opts(level='underlay') * iplot 

In [ ]:
# use a template for the Panel dashboard
template = pn.template.FastListTemplate(
    title='Interactive Panel Dashboard with Snowflake', 
    sidebar=[select_amenity],
    main=[(map_tiles.opts(level='underlay') * iplot).panel()],
    accent_base_color="#ff6f69",
    header_background="#ff6f69",
    theme="dark"
)
template.show()
# template.servable();